## 完整预测流程
### 1.导入必要的包

In [2]:
import json
import numpy as np
import pandas as pd

from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense
from tools.Attention import Attention
from keras.layers import GRU, Bidirectional
from tqdm import tqdm
from tkinter import _flatten

from pandarallel import pandarallel 
from keras.models import load_model
import benchmark_evaluation as eva
import config as cfg
import sys
sys.path.append("../tools/")
import tools.funclib as funclib

pandarallel.initialize() 

INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# embeddin 方法
EMBD_METHO='esm32'
EMBD_WH = 1280

embd_feature = pd.read_feather(f"/home/dengrui/DMLF/data/featureBank/embd_{EMBD_METHO}.feather")

ecs_2 = [True, False]
ecs_8 = [1,2,3,4,5,6,7,8]
label_multi = pd.read_feather('/home/dengrui/anaconda3/PyProjects/MutiLabelProjects/multi-label-classification-4-event-type/multi-label-classification-4-event-type/label_multi.feather')
ecs_multi = label_multi['label_multi'].tolist()

## 读取测试数据

In [4]:
# 读取测试数据
test = pd.read_csv('/home/dengrui/DMLF/data/corynebacterium.tsv',sep='\t')[['id','seq','ec']]


# embeddin 方法
EMBD_METHO='esm32'
EMBD_WH = 1280

data = pd.read_feather(f"/home/dengrui/DMLF/data/featureBank/embd_{EMBD_METHO}.feather")
test= test.merge(data, on='id', how='left')
test_feature = test.iloc[:, 3:]
test_feature = np.array(test_feature)
test_feature =np.reshape(test_feature,(test_feature.shape[0],1,EMBD_WH))

# 测试结果
test_res= test[['id','seq', 'ec']].copy()


In [4]:
# task3_test
data_task3_test_2019 = pd.read_feather(cfg.FILE_TASK3_TEST_2019)
data_task3_test_2020 = pd.read_feather(cfg.FILE_TASK3_TEST_2020)
data_task3_test_2021 = pd.read_feather(cfg.FILE_TASK3_TEST_2021)
data_task3_test_2022 = pd.read_feather(cfg.FILE_TASK3_TEST_2022)
data_t3_all_set = pd.concat([data_task3_test_2019,data_task3_test_2020,data_task3_test_2021,data_task3_test_2022])
test = data_t3_all_set.drop_duplicates().reset_index(drop=True)
test =test.rename(columns={'ec_number':'ec'})

# embeddin 方法
EMBD_METHO='esm32'
EMBD_WH = 1280


test = test.merge(embd_feature, on='id', how='left')
test_feature = test.iloc[:,3:]
test_feature = np.array(test_feature)
test_feature =np.reshape(test_feature,(test_feature.shape[0],1,EMBD_WH))


test_res= test[['id','seq', 'ec']].copy()

# task1

In [5]:
from keras.models import load_model
listec = list()
load_model = load_model("/home/dengrui/anaconda3/PyProjects/MutiLabelProjects/multi-label-classification-4-event-type/multi-label-classification-4-event-type/task1_2022.h5",custom_objects={"Attention": Attention},compile=False)
predicted = load_model.predict(test_feature)
test_res['pred_isenzyme'] = pd.DataFrame(np.argmax(predicted, axis=-1)).astype(bool)

2022-11-22 05:57:32.639181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 05:57:33.250175: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46702 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:86:00.0, compute capability: 8.6


 35/164 [=====>........................] - ETA: 0s  

2022-11-22 05:57:36.461637: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2022-11-22 05:57:36.609257: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


164/164 [==============================] - 2s 1ms/step


# taks2

In [6]:
from keras.models import load_model

listec = list()
load_model = load_model("/home/dengrui/anaconda3/PyProjects/MutiLabelProjects/multi-label-classification-4-event-type/multi-label-classification-4-event-type/task2_model_2022_label8.h5",custom_objects={"Attention": Attention},compile=False)
predicted = load_model.predict(test_feature)
counter = 0
for i in tqdm(range(len(test))):
    listec.append(sorted(dict(zip(set(ecs_8),predicted[i])).items(),key = lambda x:x[1], reverse = True)[0][0])

test_res['pred_functions']=listec

164/164 [==============================] - 1s 1ms/step


100%|██████████| 5229/5229 [00:00<00:00, 352236.66it/s]


# task3

In [7]:
def make_rec_str(pred_top):
    pred = [item[0]+'('+str(item[1])+')' for item  in pred_top]
    pred = ', '.join(pred)
    return pred

def make_multi_ec_pred(pred_func, pred_top_list):
    res_ecs = pred_top_list[:pred_func]
    res_ecs = [item[0] for item in res_ecs]
    return ','.join(res_ecs)

from keras.models import load_model
listec = list()
listpred = list()
model = load_model(f'/home/dengrui/anaconda3/PyProjects/MutiLabelProjects/multi-label-classification-4-event-type/multi-label-classification-4-event-type/task3_{EMBD_METHO}_2022.h5',custom_objects={"Attention": Attention},compile=False)
predicted = model.predict(test_feature)
counter = 0
for i in tqdm(range(len(test_feature))):
    listpred.append(sorted(dict(zip((ecs_multi),predicted[i])).items(),key = lambda x:x[1], reverse = True)[0][0])
    listend= list()
    sortlist = sorted(dict(zip((ecs_multi),predicted[i])).items(),key = lambda x:x[1], reverse = True)
    # listpred.append(sortlist[0])
    for j in range(0,len(sortlist)):
        if j == 20:
            break
        listend.append([sortlist[j][0], round(sortlist[j][1],4)])
    listec.append(listend)

test_res['pred_ec']=listpred
test_res['pred_top20'] = listec
test_res['pred_ec'] = test_res.apply(lambda x : x.pred_ec if x.pred_functions <=1 else make_multi_ec_pred(pred_func=x.pred_functions, pred_top_list=x.pred_top20), axis=1)
test_res['pred_top20'] = test_res['pred_top20'].apply(lambda x : make_rec_str(pred_top=x))
test_res['pred_ec'] = test_res.apply(lambda x : x.pred_ec if x.pred_isenzyme ==True else '-', axis=1)
test_res

164/164 [==============================] - 1s 1ms/step


100%|██████████| 5229/5229 [00:23<00:00, 223.30it/s]


,id,seq,ec,pred_isenzyme,pred_functions,pred_ec,pred_top20
0,L8EYU3,MPFTQKEITYLRAQGYGRLATVGAHGEPHNVPVSFEIDEERGTIEI...,1.3.98.4,False,1,-,"1.-.-.-(0.8454), 1.6.5.-(0.2256), 4.2.1.96(0.0..."
1,L8EUQ6,MRYDVVIAGAGPTGLMLACELRLAGARTLVLERLAEPVDFSKALGV...,"1.14.13.38,1.14.13.234",True,1,1.14.13.38,"1.14.13.38(0.1901), 1.14.13.-(0.1413), 1.14.13..."
2,I7HUG0,MVQLTIDNARGVTLCRVSLPANATVQQLLLQLTVAKPELRQAQAIR...,3.4.23.-,True,1,3.4.23.-,"3.4.23.-(0.9962), 3.1.3.16(0.0091), 2.7.7.7(0...."
3,P0DPE4,MAILQIGAGGVGWVVAHKAAQNNDVLGDITIASRTVGKCEKIIESI...,1.5.1.43,True,1,2.5.1.44,"2.5.1.44(0.1729), 1.5.1.43(0.1085), 1.2.1.59(0..."
4,P0DPF0,MPDNDSHSLKSLPERQREDLFSAGSPSLEARKKQLSRLKTMIVDHE...,1.2.99.10,True,1,1.2.1.3,"1.2.1.3(0.1494), 1.2.1.68(0.0479), 1.2.1.-(0.0..."
...,...,...,...,...,...,...,...
5224,A2WLP4,MVSAAAGWAAPAFAVAAVVIWVVLCGELLRRRRRGAGSGKGDAAAA...,1.14.14.-,True,1,1.14.99.-,"1.14.99.-(0.3041), 1.14.-.-(0.2485), 1.14.13.9..."
5225,A0A250YGJ5,MSVNYAAGLSPYADKGKCGLPEIFDPPEELERKVWELARLVRQSSN...,"2.4.2.-,2.3.1.286",True,1,3.5.1.-,"3.5.1.-(0.999), 2.4.2.-(0.1939), 4.2.99.-(0.00..."
5226,A0A2K5TU92,MSVNYAAGLSPYADKGKCGLPEIFDPPEELERKVWELARLVWQSSH...,"2.4.2.-,2.3.1.286",True,1,3.5.1.-,"3.5.1.-(0.9994), 2.4.2.-(0.0987), 4.2.99.-(0.0..."
5227,A0A3R0A696,MKHWKKMAASLIAISTMVAVVPTTYAMESEDSQPQTTDTATVQTTK...,3.2.1.55,True,1,3.4.21.110,"3.4.21.110(0.1582), 2.4.2.-(0.0659), 3.2.1.28(..."


## balst

In [9]:
sprot = pd.read_feather(cfg.DATADIR+'uniprot/snap201802.feather')
test.ec = test.ec.fillna('-')
ec_sprot = _flatten([item.split(',') for item in list(set(sprot.ec_number))])


res_diamond = funclib.getblast(train=sprot[['id','seq']], test=test[['id','seq']])
res_diamond = res_diamond [['id','sseqid','pident']].merge(sprot[['id','ec_number']], how='left', left_on='sseqid', right_on='id')
res_diamond = res_diamond[['id_x', 'ec_number','pident']].rename(columns={'id_x':'id','ec_number':'ec_diamond'})
test_res = test_res.merge(res_diamond, on='id', how='left').drop_duplicates().reset_index(drop=True)
test_res.ec_diamond = test_res.ec_diamond.fillna('-')


Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


In [12]:
test_res[['id', 'pred_ec', 'pred_top20']].rename(columns={'pred_ec':'ec_ours'}).to_csv(cfg.RESULTSDIR+'case_gubang/dmlf/corynebacterium_pred.tsv', index=None, sep='\t')

In [16]:
test_res

,id,seq,ec,pred_isenzyme,pred_functions,pred_ec,pred_top20,ec_diamond,pident,predictable
0,L8EYU3,MPFTQKEITYLRAQGYGRLATVGAHGEPHNVPVSFEIDEERGTIEI...,1.3.98.4,False,1,-,"1.-.-.-(0.8454), 1.6.5.-(0.2256), 4.2.1.96(0.0...",-,NaN,False
1,L8EUQ6,MRYDVVIAGAGPTGLMLACELRLAGARTLVLERLAEPVDFSKALGV...,"1.14.13.38,1.14.13.234",True,1,1.14.13.38,"1.14.13.38(0.1901), 1.14.13.-(0.1413), 1.14.13...",1.14.13.38,99.2,False
2,I7HUG0,MVQLTIDNARGVTLCRVSLPANATVQQLLLQLTVAKPELRQAQAIR...,3.4.23.-,True,1,3.4.23.-,"3.4.23.-(0.9962), 3.1.3.16(0.0091), 2.7.7.7(0....",3.4.23.-,46.1,True
3,P0DPE4,MAILQIGAGGVGWVVAHKAAQNNDVLGDITIASRTVGKCEKIIESI...,1.5.1.43,True,1,2.5.1.44,"2.5.1.44(0.1729), 1.5.1.43(0.1085), 1.2.1.59(0...",1.5.1.43,84.1,True
4,P0DPF0,MPDNDSHSLKSLPERQREDLFSAGSPSLEARKKQLSRLKTMIVDHE...,1.2.99.10,True,1,1.2.1.3,"1.2.1.3(0.1494), 1.2.1.68(0.0479), 1.2.1.-(0.0...",1.2.1.3,42.0,False
...,...,...,...,...,...,...,...,...,...,...
5226,A2WLP4,MVSAAAGWAAPAFAVAAVVIWVVLCGELLRRRRRGAGSGKGDAAAA...,1.14.14.-,True,1,1.14.99.-,"1.14.99.-(0.3041), 1.14.-.-(0.2485), 1.14.13.9...",1.14.-.-,99.2,True
5227,A0A250YGJ5,MSVNYAAGLSPYADKGKCGLPEIFDPPEELERKVWELARLVRQSSN...,"2.4.2.-,2.3.1.286",True,1,3.5.1.-,"3.5.1.-(0.999), 2.4.2.-(0.1939), 4.2.99.-(0.00...",3.5.1.-,87.3,False
5228,A0A2K5TU92,MSVNYAAGLSPYADKGKCGLPEIFDPPEELERKVWELARLVWQSSH...,"2.4.2.-,2.3.1.286",True,1,3.5.1.-,"3.5.1.-(0.9994), 2.4.2.-(0.0987), 4.2.99.-(0.0...",3.5.1.-,93.5,False
5229,A0A3R0A696,MKHWKKMAASLIAISTMVAVVPTTYAMESEDSQPQTTDTATVQTTK...,3.2.1.55,True,1,3.4.21.110,"3.4.21.110(0.1582), 2.4.2.-(0.0659), 3.2.1.28(...",-,NaN,True


In [21]:
def make_ec_predictable_label(in_ec):
    in_ecs = set(in_ec.split(','))
    intersection = in_ecs.intersection(set(ecs_multi))
    if len(in_ecs) == len(intersection):
        return True
    else:
        return False
        
test_res['pred_ec']=test_res.apply(lambda x: x.ec_diamond if x.ec_diamond!='-' else x.pred_ec, axis=1)
test_res['predictable']=test_res.ec.parallel_apply(lambda x : make_ec_predictable_label(in_ec=x))

In [26]:
test_res[test_res.ec==test_res.pred_ec]

,id,seq,ec,pred_isenzyme,pred_functions,pred_ec,pred_top20,ec_diamond,pident,predictable
2,I7HUG0,MVQLTIDNARGVTLCRVSLPANATVQQLLLQLTVAKPELRQAQAIR...,3.4.23.-,True,1,3.4.23.-,"3.4.23.-(0.9962), 3.1.3.16(0.0091), 2.7.7.7(0....",3.4.23.-,46.1,True
3,P0DPE4,MAILQIGAGGVGWVVAHKAAQNNDVLGDITIASRTVGKCEKIIESI...,1.5.1.43,True,1,1.5.1.43,"2.5.1.44(0.1729), 1.5.1.43(0.1085), 1.2.1.59(0...",1.5.1.43,84.1,True
5,Q6EZC2,MLKILWTYILFLLFISASARAEKPWYFDAIGLTETTMSLTDKNTPV...,3.4.21.-,True,1,3.4.21.-,"3.4.21.-(0.9992), 3.4.21.62(0.0081), 2.1.1.77(...",-,NaN,True
6,O51767,MNDFKLPIYKYKDELIKVLKNHNVLIVESPTGSGKTTQLPRIIYEA...,3.6.4.13,True,1,3.6.4.13,"3.6.4.13(0.9974), 3.6.4.-(0.0187), 3.6.3.-(0.0...",3.6.4.13,39.6,True
12,Q8RXP6,MVTYVSALFYGFGGIVVAGVALLVAFQEKLVYVPVLPGLSKSYPIT...,3.-.-.-,True,1,3.-.-.-,"3.-.-.-(0.9244), 3.1.1.23(0.05), 3.1.2.22(0.01...",3.-.-.-,36.9,True
...,...,...,...,...,...,...,...,...,...,...
5212,F4JZJ0,MLSPRRFILVVTILGLLGHNLTHVLNSPHRILLDTDVDTDDFIALL...,3.2.2.-,True,1,3.2.2.-,"3.2.2.-(0.0068), 3.2.2.3(0.0048), 3.1.3.2(0.00...",-,NaN,True
5213,P0DV82,SLFEFGKMILKETGKNPFTSYGFYGCYCGWGGRGGPKDATDRCCF,3.1.1.4,True,1,3.1.1.4,"3.1.1.4(0.9995), 3.1.1.32(0.0013), 3.2.1.14(0....",3.1.1.4,82.2,True
5214,Q8CIH9,MELEELGIREECGVFGCIASGDWPTQLDVPHVITLGLVGLQHRGQE...,2.4.2.14,True,1,2.4.2.14,"2.4.2.14(0.9891), 2.6.1.16(0.0135), 6.3.5.4(0....",2.4.2.14,96.9,True
5218,J4VV61,MSPSRSEPLIDNLVDRSNLKIVALASPADGHTFPLLRIVEELVLRG...,2.4.1.-,True,1,2.4.1.-,"2.4.1.-(0.9662), 2.4.1.318(0.0279), 4.3.3.5(0....",2.4.1.-,34.5,True


In [30]:
2390/4373

0.5465355591127372

In [31]:
test_res[(test_res.predictable)&(~test_res.ec.str.contains('-'))]

,id,seq,ec,pred_isenzyme,pred_functions,pred_ec,pred_top20,ec_diamond,pident,predictable
3,P0DPE4,MAILQIGAGGVGWVVAHKAAQNNDVLGDITIASRTVGKCEKIIESI...,1.5.1.43,True,1,1.5.1.43,"2.5.1.44(0.1729), 1.5.1.43(0.1085), 1.2.1.59(0...",1.5.1.43,84.1,True
6,O51767,MNDFKLPIYKYKDELIKVLKNHNVLIVESPTGSGKTTQLPRIIYEA...,3.6.4.13,True,1,3.6.4.13,"3.6.4.13(0.9974), 3.6.4.-(0.0187), 3.6.3.-(0.0...",3.6.4.13,39.6,True
7,Q845S9,MSTPLNVVALSGGTSRPSRTLALTEAILAELAEHLHIKPHLIELGE...,1.5.1.42,True,1,1.5.1.38,"1.5.1.38(0.8984), 1.6.5.2(0.1089), 1.7.-.-(0.0...",1.5.1.38,62.9,True
9,Q0WQX9,MVFGWRKAFCTSVSSNQDKPQQHSSLHTTDPPIPTPRFRSKFGFLS...,2.3.2.27,False,1,-,"2.3.2.27(0.8506), 3.1.3.16(0.2725), 2.3.2.23(0...",-,NaN,True
10,Q9ZQ46,MLNGWRRAFCTSIPKETNQNDVDDDGLVGLRHKSTSRFGFFSTPST...,2.3.2.27,False,1,-,"2.3.2.27(0.9265), 3.1.3.16(0.1768), 2.7.12.1(0...",-,NaN,True
...,...,...,...,...,...,...,...,...,...,...
5209,B9K712,MKMLYDLAKKRKTVRRFKKEKPPLEDLIYSLKVANEAPSGMNAQPW...,1.21.1.1,True,1,1.22.-.-,"1.-.-.-(0.9929), 1.6.99.3(0.0389), 1.21.1.1(0....",1.22.-.-,30.9,True
5213,P0DV82,SLFEFGKMILKETGKNPFTSYGFYGCYCGWGGRGGPKDATDRCCF,3.1.1.4,True,1,3.1.1.4,"3.1.1.4(0.9995), 3.1.1.32(0.0013), 3.2.1.14(0....",3.1.1.4,82.2,True
5214,Q8CIH9,MELEELGIREECGVFGCIASGDWPTQLDVPHVITLGLVGLQHRGQE...,2.4.2.14,True,1,2.4.2.14,"2.4.2.14(0.9891), 2.6.1.16(0.0135), 6.3.5.4(0....",2.4.2.14,96.9,True
5221,Q581T1,MRAIRLTVACRYLGPFRSVTLSPVVLPVRLFQTQEITPGEVSLVEA...,3.6.4.13,True,1,3.6.4.13,"3.6.4.13(0.7888), 2.7.7.-(0.0802), 3.1.26.-(0....",3.6.4.13,34.5,True


In [23]:
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=test_res[(test_res.predictable)&(~test_res.ec.str.contains('-'))].ec, predict=test_res[(test_res.predictable)&(~test_res.ec.str.contains('-'))].pred_ec, baselineName='ours', type='multi')

 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
        ours  		0.603805  	0.738192 		0.662757 	0.452271
